<a href="https://colab.research.google.com/github/tgwon/ai4ml/blob/main/week2/3-making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02


In [8]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [9]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [ ]:
earthquake_json['metadata']

{'generated': 1604267813000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-10-01&endtime=2020-10-31',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.10.3',
 'count': 13706}

In [10]:
earthquake_json['type']

'FeatureCollection'

In [11]:
earthquake_json['bbox']

[-179.9878, -63.765, -3.24, 179.9884, 79.5583, 669.982]

In [ ]:
#earthquake_json['features']

Each element in the JSON array `features` is a row of data for our dataframe.

In [13]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [14]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.8,
  'place': '7 km SSE of Ferry, Alaska',
  'time': 1697500524871,
  'updated': 1697500656092,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023dacc3xj',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023dacc3xj&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 50,
  'net': 'ak',
  'code': '023dacc3xj',
  'ids': ',ak023dacc3xj,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.58,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.8 - 7 km SSE of Ferry, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-149.0309, 63.956, 0]},
 'id': 'ak023dacc3xj'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [17]:
[quake['properties'] for quake in earthquake_json['features']][0]

{'mag': 1.8,
 'place': '7 km SSE of Ferry, Alaska',
 'time': 1697500524871,
 'updated': 1697500656092,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023dacc3xj',
 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023dacc3xj&format=geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 50,
 'net': 'ak',
 'code': '023dacc3xj',
 'ids': ',ak023dacc3xj,',
 'sources': ',ak,',
 'types': ',origin,phase-data,',
 'nst': None,
 'dmin': None,
 'rms': 0.58,
 'gap': None,
 'magType': 'ml',
 'type': 'earthquake',
 'title': 'M 1.8 - 7 km SSE of Ferry, Alaska'}

In [18]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.80,"7 km SSE of Ferry, Alaska",1697500524871,1697500656092,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023dacc3xj,",",ak,",",origin,phase-data,",NaN,NaN,0.58,NaN,ml,earthquake,"M 1.8 - 7 km SSE of Ferry, Alaska"
1,2.02,"1 km SW of Pāhala, Hawaii",1697500335030,1697500536060,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73610922,",",hv,",",origin,phase-data,",36.0,NaN,0.12,157.0,md,earthquake,"M 2.0 - 1 km SW of Pāhala, Hawaii"
2,1.70,"14 km W of Susitna North, Alaska",1697500319616,1697500408585,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023dacbbqx,",",ak,",",origin,phase-data,",NaN,NaN,0.48,NaN,ml,earthquake,"M 1.7 - 14 km W of Susitna North, Alaska"
3,2.20,"0 km ESE of Clio, CA",1697500104320,1697545213388,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73948100,nn00867459,",",nc,nn,",",nearby-cities,origin,phase-data,scitech-link,",17.0,0.1047,0.10,91.0,md,earthquake,"M 2.2 - 0 km ESE of Clio, CA"
4,2.23,"12 km S of Honoka‘a, Hawaii",1697500088540,1697501468930,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73610917,",",hv,",",origin,phase-data,",40.0,NaN,0.15,142.0,ml,earthquake,"M 2.2 - 12 km S of Honoka‘a, Hawaii"


### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>